## Below is the code for scraping customer reviews on HomeAway's website using Selenium in Python

HomeAway is a vacation rental marketplace with more than 2,000,000 vacation rentals in 190 countries listed on its website (https://www.homeaway.com/)

In [1]:
from time import sleep
from selenium import webdriver
from string import punctuation
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import copy
import matplotlib.pyplot as plt
%pylab inline
import numpy as np
import pandas as pd
from time import sleep

Populating the interactive namespace from numpy and matplotlib


C:\Users\bandi\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
driver = webdriver.Chrome('C:/Users/bandi/Desktop/Text Analytics/chromedriver_win32/chromedriver')
# Below is the code to open himeaway.com's homes in Austin,Texas US
property_links = []
for i in range(1,4):
    url ='https://www.homeaway.com/results/page:' + str(i) +'/region:7116?petIncluded=false&ssr=true&orderBy=averageRating&ascending=true'
    driver.get(url)
    #sleep(1)
    links = driver.find_elements_by_xpath("//a[@href]")
    for link in links:
        property_links.append(link.get_attribute('href'))       

In [9]:
link_list = []
for link in property_links:
    q = re.findall ("/p", link)
    p = re.findall("vacation-rental",link)
    if q !=[] and p!=[]:
        link_list.append(link)
link_list_unique = np.unique(link_list)
np.savetxt("link_list.csv", link_list_unique, delimiter=",", fmt='%s', header='Link')
link_list_ll = link_list_unique[52:]

In [ ]:
df_next = pd.DataFrame(columns = ['Property Link','Date','Review Rating','Review Tagline','Comments'])
for link in link_list_ll:

    url = link + "#reviews"
    #open new property reviews page 
    driver.get(url)
    #print(url)
    sleep(3)
    temp =1
    while (temp ==1):
        #get all ids on that page
        ids = driver.find_elements_by_xpath("//*[@id]")
        ##get all comments by showing all reviews
        comment_ids = []
        #make a list of all the ids
        for i in ids:       
            comment_ids.append(i.get_attribute('id'))
    
        actual_ids = []
        #get tha actual comment ids
        for x in comment_ids:
            z = re.findall ("-", x)
            if len (x) == 36 and z!=[]:
                actual_ids.append(x)
        if actual_ids == []:
            temp = 0
        #print(actual_ids)
        count = 0
        #for each comment get attributes
        for aid in actual_ids:
            sleep(3)
            #extract review tagline
            review_tagline = driver.find_elements_by_xpath('//*[@id="' + aid +'"]/div/h4')[0]
            tagline= review_tagline.get_attribute('innerText')
            #extract review rating
            review_rating = driver.find_elements_by_xpath('//*[@id="' + aid +'"]/div/div[1]/span[1]/span')[0] 
            rating= review_rating.get_attribute('innerText')
            #review comment
            review_comment = driver.find_elements_by_xpath('//*[@id="' + aid +'"]/div/div[3]')[0]
            comment= review_comment.get_attribute('innerText')
            #extract review date
            review_date = driver.find_elements_by_xpath('//*[@id="' + aid+ '"]/div/div[4]/div/span[2]')[0]
            date= review_date.get_attribute('innerText')
            
            #add review to df
            df_next.loc[len(df_next)] = [url,date, rating, tagline, comment]
            count +=1
            #print("Count of review is", count)
            
            click_button = driver.find_elements_by_xpath('//*[@id="reviews"]/div/section/div/div[2]/div[1]/button[2]')
            if click_button != []:
                click_button = driver.find_elements_by_xpath('//*[@id="reviews"]/div/section/div/div[2]/div[1]/button[2]')[0]
                check = click_button.get_property('disabled')
                if count ==6:
                    if check == False:
                        click_button.click()
                        #print('Moved to next page')
                        sleep(4)
                    else:
                        temp = 0
                        break
                else:
                     if count == len(actual_ids):
                        #print('break out of while loop when no next page')
                        temp = 0
                        break
            else:
                if count == len(actual_ids):
                    #print("Only a singe page: No more reviews")
                    temp =0
        

Export Property Link, Date, Review Rating, Tagline and Comments into a csv

In [17]:

df.to_csv('comments_homeaway.csv', header=True, sep=',') 

# For a single link

In [161]:
driver = webdriver.Chrome('C:/Users/teeru/Downloads/chromedriver.exe')
driver.get("https://www.homeaway.com/vacation-rental/p1063042vb#reviews")
sleep(2)

In [162]:
df_new = pd.DataFrame(columns = ['Property Link','Date','Review Rating','Review Tagline','Comments'])
temp =1
while (temp ==1):
    driver.implicitly_wait(30)
    #get all ids on that page
    ids = driver.find_elements_by_xpath("//*[@id]")
    ##get all comments by showing all reviews
    comment_ids = []
    #make a list of all the ids
    for i in ids:       
        comment_ids.append(i.get_attribute('id'))
    actual_ids = []
    #get tha actual comment ids
    for x in comment_ids:
        if len (x) == 36:
            actual_ids.append(x)
        #for each comment get attributes
    for aid in actual_ids:
        #extract review rating
        review_rating = driver.find_elements_by_xpath('//*[@id="' + aid +'"]/div/div[1]/span[1]/span')[0] 
        rating= review_rating.get_attribute('innerText')
        #extract review date
        review_date = driver.find_elements_by_xpath('//*[@id="' + aid+ '"]/div/div[4]/div/span[2]')[0]
        date= review_date.get_attribute('innerText')
        #extract review tagline
        review_tagline = driver.find_elements_by_xpath('//*[@id="' + aid +'"]/div/h4')[0]
        tagline= review_tagline.get_attribute('innerText')
        #review comment
        review_comment = driver.find_elements_by_xpath('//*[@id="' + aid +'"]/div/div[3]')[0]
        comment= review_comment.get_attribute('innerText')
        #add review to df
        df_new.loc[len(df_new)] = ["https://www.homeaway.com/vacation-rental/p4866864#reviews",date, rating, tagline, comment]
            
    click_button = driver.find_elements_by_xpath('//*[@id="reviews"]/div/section/div/div[2]/div[1]/button[2]')
    if click_button!= []:
        click_button = driver.find_elements_by_xpath('//*[@id="reviews"]/div/section/div/div[2]/div[1]/button[2]')[0]
        check = click_button.get_property('disabled')
        if check == False:
            click_button.click()
            sleep(5)
        else:
            temp = 0
    else:
        temp =0
            

#### Below code is used to scrape property headline, price, area, property description and amenities. We scraped this for 207 properties which had about 8000 total customer reviews.

In [4]:
property_links = pd.read_csv('property links.csv',header = None)

In [ ]:
property_descriptions = pd.DataFrame(columns = ['property_url','headline','price','area','description','amenities'])

for i in range(len(property_links)):
    property_url = property_links.iloc[i][0]
    driver.get(property_url.replace('#reviews',''))
    
    headline = driver.find_elements_by_xpath('//*[@id="overview"]/div[1]/div/div[2]/div/div[1]/h1')[0].text
    price = driver.find_elements_by_xpath('//*[@id="root"]/div/div[1]/main/div[1]/div[2]/div/div/div[2]/div[2]/div/div[1]/div[1]/div/div/div/span[1]')[0].text
    average_area = driver.find_elements_by_xpath('//*[@id="overview"]/div[1]/section/div/div/div[1]/div/ul[1]/li[1]/span[2]')[0].text
    property_description = driver.find_elements_by_xpath('//*[@id="property-description"]/div/div/div/p[1]')[0].text
    amenities = driver.find_elements_by_xpath('//*[@id="amenities"]/div[1]/div/div/div/div[1]/ul')[0].text
    
    property_descriptions.loc[len(property_descriptions)] = [property_url,headline,price,average_area,\
                                                             property_description,amenities]

In [ ]:
# Making a copy of propert_desciptions data frame
import copy
property_descriptions_copy = copy.deepcopy(property_descriptions)
def remove_space(s):
    return s.replace("\n",", ")
property_descriptions_copy['amenities'] = property_descriptions_copy['amenities'].apply(remove_space)

In [ ]:
# Exporting the data ot a csv
property_descriptions_copy.to_csv('property_descriptions.csv', header=True, sep=',') 